# Env Set Up #

In [2]:
# Import libraries
import os
import cv2
import time
import random
import gc as G
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from numba import cuda
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras import backend as K, mixed_precision
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from concurrent.futures import ThreadPoolExecutor
import keras_tuner as kt
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau as RLOP, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier as KC
from sklearn.model_selection import RandomizedSearchCV as RSCV
from utils import import_train, view_train_images, plot_training_results, fast_import as FI, fast_import2 as FI2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization


In [3]:
# # limit usage of GPU memory
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)  # Prevents TensorFlow from allocating all memory at once
#             print(f"Enabled memory growth for: {gpu}")

#         # Set GPU device
#         device = "/GPU:0"
#     except RuntimeError as e:
#         print(e)
# else:
#     device = "/CPU:0"
#     print("No GPU detected, using CPU.")

# print(f"Using device: {device}")

# # set gpu to use max memory
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.set_logical_device_configuration(
#                 gpu,
#                 [tf.config.LogicalDeviceConfiguration(memory_limit=3072)]  # Set in MB (e.g., 10GB)
#             )
#         print("Set TensorFlow to use max GPU memory.")
#     except RuntimeError as e:
#         print(e)


# 1) Data Import #

In [4]:
# paths
train_dir = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/train"
val_dir = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/val"
test_dir = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Data/CLS-LOC/test"

# path to save tuner results
kt_path = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Tuning_Res/KT"

# save best model
model_path = "C:/Users/RoiMinuit/Desktop/data/ILSVRC/Tuning_Res/KT/Model"


In [5]:
# data import - 35K
train_images, train_labels = FI2(train_dir, 25000)
test_images, test_labels = FI2(val_dir, 5000)

Function processed 25000 images in 27 seconds.

Function processed 5000 images in 8 seconds.



In [6]:
# prepare training data
X_train = np.array(train_images)
X_train = np.repeat(X_train, 3, -1)

Y_train = [str(s) for s in train_labels]
label_encoder = LabelEncoder()
Y_train = label_encoder.fit_transform(Y_train)
Y_hot = to_categorical(Y_train, num_classes=1000)


In [7]:
# prepare testing data
X_test = np.array(test_images)
X_test = np.repeat(X_test, 3, -1)

Y_test = [str(s) for s in test_labels]
label_encoder = LabelEncoder()
Y_test = label_encoder.fit_transform(Y_test)
Y_test_hot = to_categorical(Y_test, num_classes=1000)


In [8]:
# check data shape
print(Y_train.shape)
print(Y_test.shape)

print(len(np.unique(Y_train)))
print(len(np.unique(Y_test)))

(25000,)
(5000,)
1000
1


# 2) Model Optimization #

## a) Keras Classifier ##

In [ ]:
# parameters for tuning
epx = 50
batch_size = 8
RSCV_opts = ['adam', 'rmsprop']
krnl = (3, 3)
in_shape = (64, 64, 3)
RSCV_activation = ['relu', 'tanh', 'sigmoid']
RSCV_node1 = [2, 4, 8, 16, 32]
RSCV_node2 = [2, 4, 8, 16, 32]
RSCV_drop = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
RSCV_bacth = [2, 4, 6, 8, 16, 32, 64, 128]

params_grid = dict(node1 = RSCV_node1,
               node2 = RSCV_node2,
               activations = RSCV_activation,
               optimizers = RSCV_opts,
               batch_size = RSCV_bacth,
               drops = RSCV_drop)

In [ ]:
# Keras Classifier needs a model builder function to instantiate a new model for each new grid point
def build_CNN(optimizers, node1, activations, node2, drops):
    CNN = Sequential([
        # first layer
        Conv2D(node1, kernel_size=krnl, activation=activations, input_shape=in_shape),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        # 2nd layer
        Conv2D(node2, kernel_size=krnl, activation=activations),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        # output layer
        GlobalAveragePooling2D(),
        Dense(drops),
        Dense(1000, 'softmax')
    ])
    CNN.compile(optimizers,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    # return built model
    return CNN

In [ ]:
# build Keras Classifier
RS_cnn = KC(build_fn=build_CNN,
            verbose=1,
            epochs=5,
            batch_size=batch_size)

In [ ]:
# establish search space and search
grid_search = RSCV(RS_cnn,
                   param_distributions=params_grid,
                   cv=5,
                   scoring='accuracy',
                   verbose=1)

# execute search with categorical Y, not one-hotted
grid_search.fit(X_train, Y_train)

## b) Keras Tuner ##

In [45]:
# variables for tuning
activations = ['relu', 'leaky_relu', 'selu', 'elu']
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint(model_path + '/ImgNet_Model.h5', monitor='val_loss', save_best_only=True, save_freq='epoch')

In [46]:
# base model for kt
def build_KT_CNN(hp):
    with tf.device('/GPU:0'):
        layers = [
            # input layer
            Conv2D(hp.Int('filters', min_value=32, max_value=128, step=32),
                input_shape=(64, 64, 3),
                kernel_size=(3, 3),
                activation=hp.Choice('activation', activations)),
                Dropout(hp.Float('drop_rate', 1e-5, 1e-2))
        ]
        
        # hidden layer(s)
        for l in range(hp.Int('num_layers', 1, 3)):
            layers.append(Conv2D(hp.Int('filters', min_value=2, max_value=64),
                                kernel_size=(3, 3),
                                activation=hp.Choice('activation', activations)))
        
        # output layer
        layers.append(GlobalAveragePooling2D())
        layers.append(Dropout(hp.Float('drop_rate', 1e-4, 1e-1)))
        layers.append(Dense(1000, activation=hp.Choice('activation', activations)))
        
        CNN = Sequential(layers)
        lr = hp.Float('learning_rate', min_value=1e-5, max_value=1e-1, sampling='LOG', default=1e-3)
        CNN.compile(optimizer=RMSprop(learning_rate=lr),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
        # return built model
        return CNN

build_KT_CNN(kt.HyperParameters())


In [48]:
# eastablish search space and search
tuner = kt.RandomSearch(
    hypermodel=build_KT_CNN,
    objective='val_accuracy',
    max_trials=100,
    executions_per_trial=4,
    overwrite=False,
    directory=kt_path,
    project_name='ImgNetOptimization'
)

# search
tuner.search(X_train, Y_train, epochs=20, validation_data=(X_test, Y_test), callbacks=[early_stop, checkpoint])
tuner.results_summary()


Reloading Tuner from C:/Users/RoiMinuit/Desktop/data/ILSVRC/Tuning_Res/KT\ImgNetOptimization\tuner0.json

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
128               |32                |filters
relu              |relu              |activation
7.32e-05          |0.0043041         |drop_rate
1                 |2                 |num_layers
2.7424e-05        |5.6172e-05        |learning_rate

Epoch 1/20


Traceback (most recent call last):
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\hypermodel.py", line 149, in fit
    return model.fit(*args, **kwargs)
  File "c:\User

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "c:\Users\RoiMinuit\miniconda3\envs\tf\lib\site-packages\keras_tuner\src\engine\hypermodel.py", 

In [ ]:
# view optimized model
model = tuner.get_best_models()[0]
model.summary()

In [1]:
K.clear_session()
G.collect()

NameError: name 'K' is not defined